# Catmull--Rom (Recursive)

Recursive algorithm developed by Barry and Goldman (1988), according to Yuksel et al. (2011), figure 3.

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
x_1, x0, x1, x2 = sp.symbols('xbm_-1 xbm:3')

In [ ]:
t, t_1, t0, t1, t2 = sp.symbols('t t_-1 t:3')

In [ ]:
p_10 = NamedExpression('pbm_-1,0', x_1 * (t0 - t) / (t0 - t_1) + x0 * (t - t_1) / (t0 - t_1))
p_10

In [ ]:
p01 = NamedExpression('pbm_0,1', x0 * (t1 - t) / (t1 - t0) + x1 * (t - t0) / (t1 - t0))
p01

In [ ]:
p12 = NamedExpression('pbm_1,2', x1 * (t2 - t) / (t2 - t1) + x2 * (t - t1) / (t2 - t1))
p12

In [ ]:
p_101 = NamedExpression('pbm_-1,0,1', p_10.name * (t1 - t) / (t1 - t_1) + p01.name * (t - t_1) / (t1 - t_1))
p_101

In [ ]:
p012 = NamedExpression('pbm_0,1,2', p01.name * (t2 - t) / (t2 - t0) + p12.name * (t - t0) / (t2 - t0))
p012

In [ ]:
p = NamedExpression('pbm', p_101.name * (t1 - t) / (t1 - t0) + p012.name * (t - t0) / (t1 - t0))
p

In [ ]:
p = p.subs([p_101, p012]).subs([p_10, p01, p12])
p

In [ ]:
p_normalized = p.expr.subs(t, t * (t1 - t0) + t0)

In [ ]:
M_CR = NamedMatrix(
    r'{M_\text{CR}}',
    sp.Matrix([[c.expand().coeff(x).factor() for x in (x_1, x0, x1, x2)]
               for c in p_normalized.as_poly(t).all_coeffs()]))

In [ ]:
deltas = [
    (t_1, -sp.Symbol('Delta_-1')),
    (t0, 0),
    (t1, sp.Symbol('Delta0')),
    (t2, sp.Symbol('Delta0') + sp.Symbol('Delta1'))
]

In [ ]:
M_CR.simplify().subs(deltas).factor()

In [ ]:
uniform = [
    (sp.Symbol('Delta_-1'), 1),
    (sp.Symbol('Delta0') , 1),
    (sp.Symbol('Delta1') , 1),
]

In [ ]:
M_CR.subs(deltas).subs(uniform).pull_out(sp.S.Half).expr

In [ ]:
velocity = p.expr.diff(t)

In [ ]:
velocity.subs(t, t0).subs(deltas).factor()

In [ ]:
velocity.subs(t, t1).subs(deltas).factor()

in general:

\begin{equation}
\boldsymbol{\dot{x}}_i =
\frac{
(t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{
(t_{i+1} - t_i)(t_i - t_{i-1})(t_{i+1} - t_{i-1})
}
\end{equation}

You might encounter another way to write the equation for $\boldsymbol{\dot{x}}_0$
(e.g. at https://stackoverflow.com/a/23980479/):

In [ ]:
(x0 - x_1) / (t0 - t_1) - (x1 - x_1) / (t1 - t_1) + (x1 - x0) / (t1 - t0)

... but this is equivalent to the equation shown above:

In [ ]:
_.subs(deltas).factor()

Yet another way to skin this cat -- sometimes referred to as Bessel--Overhauser -- is to define the velocity of the left and right chords:

In [ ]:
v_left = (x0 - x_1) / (t0 - t_1)
v_right = (x1 - x0) / (t1 - t0)

... and then combine them in this way:

In [ ]:
((t1 - t0) * v_left + (t0 - t_1) * v_right) / (t1 - t_1)

Again, that's the same as we had above:

In [ ]:
_.subs(deltas).factor()